# Predicting Number of Flu Cases in US

I tried here; but this data is too tricky to work with, so I'm going to take a look and see if I can find a better data set.

In [6]:
# imports
import pandas as pd
import numpy as np
import os
import nibabel as nib
import matplotlib.pyplot as plt
import time
from sklearn import preprocessing

In [161]:
cdc_data = pd.read_csv('NewCDCData/WHO_NREVSS_Clinical_Labs.csv')
cdc_data.head(10)

*Beginning for the 2015-16 season, reports from public health and clinical laboratories are presented separately in the weekly influenza update, FluView. Data from clinical laboratories include the weekly total number of specimens tested, the number of positive influenza test, and the percent positive by influenza type.
REGION TYPE REGION YEAR WEEK TOTAL SPECIMENS TOTAL A TOTAL B PERCENT POSITIVE PERCENT A                                          PERCENT B                                                                                                                                                                                                                                                                               
National    X      2015 40   12029           84      43      1.05578          0.698312                                            0.357469                                                                                                                                                                                                                                                                               
                        41   13111           116     54      1.29662          0.884753                                            0.411868                                                                                                                                                                                                                                                                               
                        42   13441           97      52      1.10855          0.721672                                            0.386876                                                                                                                                                                                                                                                                               
                        43   13537           98      52      1.10807          0.723942                                            0.384132                                                                                                                                                                                                                                                                               
                        44   14687           97      68      1.12344          0.660448                                            0.462994                                                                                                                                                                                                                                                                               
                        45   15048           122     86      1.38224          0.810739                                            0.571505                                                                                                                                                                                                                                                                               
                        46   15250           84      98      1.19344          0.55082                                             0.642623                                                                                                                                                                                                                                                                               
                        47   15234           119     92      1.38506          0.781147                                            0.603912                                                                                                                                                                                                                                                                    

In [16]:
cdc_dat = pd.read_csv('NewCDCData/WHO_NREVSS_Combined_prior_to_2015_16.csv')

In [17]:
cdc_dat.head()

*Beginning for the 2015-16 season, reports from public health and clinical laboratories are presented separately in the weekly influenza update, FluView.  This data file includes only data prior to the 2015-16 influenza season, and will be presented with the public health and clinical labs combined.
REGION TYPE REGION YEAR WEEK TOTAL SPECIMENS PERCENT POSITIVE A (2009 H1N1) A (H1) A (H3) A (Subtyping not Performed) A (Unable to Subtype) B                                              H3N2v                                                                                                                                                                                                                                                          
National    X      2003 40   1430            1.53846          0             0      2      19                          0                     1                                                  0                                                                                                                                                                                                                                                          
                        41   1682            4.57788          0             0      3      74                          0                     0                                                  0                                                                                                                                                                                                                                                          
                        42   2176            11.6268          0             0      17     235                         0                     1                                                  0                                                                                                                                                                                                                                                          
                        43   2568            17.8349          0             0      56     398                         0                     4                                                  0

In [25]:
virus_vbw = pd.read_csv('VirusViewByWeek.csv')

In [26]:
virus_vbw

,Year,Week,Virus,0-4 yr,5-24 yr,25-64 yr,65+ yr
0,2019,40,A (H3),4,10,15,20
1,2019,41,A (H3),2,12,12,27
2,2019,42,A (H3),14,15,20,26
3,2019,43,A (H3),4,13,23,22
4,2019,44,A (H3),7,13,16,17
5,2019,45,A (H3),6,23,11,15
6,2019,46,A (H3),13,4,11,21
7,2019,47,A (H3),6,20,23,20
8,2019,48,A (H3),12,21,19,22
9,2019,49,A (H3),13,16,22,24


In [141]:
for col in virus_vbw.columns:
    print(col)

Year
 Week
 Virus
0-4 yr
5-24 yr
25-64 yr
65+ yr


In [28]:
virus_vbw.head(39)

,Year,Week,Virus,0-4 yr,5-24 yr,25-64 yr,65+ yr
0,2019,40,A (H3),4,10,15,20
1,2019,41,A (H3),2,12,12,27
2,2019,42,A (H3),14,15,20,26
3,2019,43,A (H3),4,13,23,22
4,2019,44,A (H3),7,13,16,17
5,2019,45,A (H3),6,23,11,15
6,2019,46,A (H3),13,4,11,21
7,2019,47,A (H3),6,20,23,20
8,2019,48,A (H3),12,21,19,22
9,2019,49,A (H3),13,16,22,24


In [155]:
num = 0
index=0
virus_1920AH3 = virus_vbw[num:(num+18)]
virus_1920AH3['sum']=virus_1920AH3['0-4 yr']+virus_1920AH3['5-24 yr']+virus_1920AH3['25-64 yr']+virus_1920AH3['65+ yr']
ah3 = virus_1920AH3.iloc[index][2]
virus_1920AH3[ah3]=virus_1920AH3['sum']
virus_1920AH3.drop([' Virus','sum','65+ yr','25-64 yr','5-24 yr','0-4 yr'],axis=1,inplace=True)
virus_1920AH3

/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.
/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


,Year,Week,A (H3)
0,2019,40,49
1,2019,41,53
2,2019,42,75
3,2019,43,62
4,2019,44,53
5,2019,45,55
6,2019,46,49
7,2019,47,69
8,2019,48,74
9,2019,49,75


In [156]:
num=num+18
virus_ = virus_vbw[num:(num+18)]
virus_['sum']=virus_['0-4 yr']+virus_['5-24 yr']+virus_['25-64 yr']+virus_['65+ yr']
a = virus_.iloc[index][2]
virus_[a]=virus_['sum']
virus_.drop([' Virus','sum','65+ yr','25-64 yr','5-24 yr','0-4 yr'],axis=1,inplace=True)
virus_

/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """


,Year,Week,A (H1N1)pdm09
18,2019,40,21
19,2019,41,30
20,2019,42,33
21,2019,43,45
22,2019,44,44
23,2019,45,85
24,2019,46,121
25,2019,47,197
26,2019,48,313
27,2019,49,414


In [159]:
new_inds = np.arange(18)
virus_ = virus_.reindex(new_inds)
virus_

,Year,Week,A (H1N1)pdm09
0,NaN,NaN,NaN
1,NaN,NaN,NaN
2,NaN,NaN,NaN
3,NaN,NaN,NaN
4,NaN,NaN,NaN
5,NaN,NaN,NaN
6,NaN,NaN,NaN
7,NaN,NaN,NaN
8,NaN,NaN,NaN
9,NaN,NaN,NaN


In [110]:
num=num+18
virus_1920AH3 = virus_vbw[num:(num+18)]
virus_1920AH3

,Year,Week,Virus,0-4 yr,5-24 yr,25-64 yr,65+ yr
180,2019,8,A (H1N1)pdm09,165,339,591,268
181,2019,9,A (H1N1)pdm09,196,265,527,244
182,2019,10,A (H1N1)pdm09,165,251,521,235
183,2019,11,A (H1N1)pdm09,129,184,452,210
184,2019,12,A (H1N1)pdm09,85,109,314,168
185,2019,13,A (H1N1)pdm09,53,83,205,98
186,2019,14,A (H1N1)pdm09,33,44,132,82
187,2019,15,A (H1N1)pdm09,19,36,84,54
188,2019,16,A (H1N1)pdm09,8,21,62,30
189,2019,17,A (H1N1)pdm09,10,12,42,20
